In [ ]:
# LightGBMRegressor
# 交叉验证 MAE: 473.3349762655072
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_absolute_error

# 首先使用 Pandas 库读入训练数据和测试数据，保存到 Train_data 和 Test_data 变量中。
Train_data = pd.read_csv('../data/used_car_train_20200313.csv',
                         sep=' ')  # handle_used_car_train.csv
Test_data = pd.read_csv('../data/used_car_testB_20200421.csv', sep=' ')

# 使用 pd.concat() 函数将训练数据和测试数据合并，并保存到 df 变量中。
df = pd.concat([Train_data, Test_data], ignore_index=True)

In [ ]:
# 对 'price' 做对数变换，使用 np.log1p() 函数。
df['price'] = np.log1p(df['price'])

In [ ]:
# 用众数填充缺失值
df['fuelType'] = df['fuelType'].fillna(0)
df['gearbox'] = df['gearbox'].fillna(0)
df['bodyType'] = df['bodyType'].fillna(0)
df['model'] = df['model'].fillna(0)

# 处理异常值，主要是将功率大于 600 的标为 600
df['power'] = df['power'].map(lambda x: 600 if x > 600 else x)  # 赛题限定power<=600
# 将 ‘notRepairedDamage’ 中的缺失值替换为 None
df['notRepairedDamage'] = df['notRepairedDamage'].astype('str').apply(lambda x: x if x != '-' else None).astype(
    'float32')

# 对可分类的连续特征进行分桶，例如将功率（power）分为 31 组，车型（model）分为 24 组。
bin = [i * 10 for i in range(31)]
df['power_bin'] = pd.cut(df['power'], bin, labels=False)

bin = [i * 10 for i in range(24)]
df['model_bin'] = pd.cut(df['model'], bin, labels=False)

In [ ]:
# 对日期数据进行处理，主要是提取年，月，日等信息和计算二手车使用时间。
from datetime import datetime


def date_process(x):
    year = int(str(x)[:4])
    month = int(str(x)[4:6])
    day = int(str(x)[6:8])

    if month < 1:
        month = 1

    date = datetime(year, month, day)
    return date


df['regDate'] = df['regDate'].apply(date_process)
df['creatDate'] = df['creatDate'].apply(date_process)
df['regDate_year'] = df['regDate'].dt.year
df['regDate_month'] = df['regDate'].dt.month
df['regDate_day'] = df['regDate'].dt.day
df['creatDate_year'] = df['creatDate'].dt.year
df['creatDate_month'] = df['creatDate'].dt.month
df['creatDate_day'] = df['creatDate'].dt.day

# 使用天数
df['car_age_day'] = (df['creatDate'] - df['regDate']).dt.days
# 使用年数
df['car_age_year'] = round(df['car_age_day'] / 365, 1)

# 对行驶路程和功率数据进行统计，例如：计算行驶路程与功率的最大值、最小值、中位数和均值等。
kk = ['kilometer', 'power']
t1 = Train_data.groupby(kk[0], as_index=False)[kk[1]].agg(
    {kk[0] + '_' + kk[1] + '_count': 'count', kk[0] + '_' + kk[1] + '_max': 'max',
     kk[0] + '_' + kk[1] + '_median': 'median',
     kk[0] + '_' + kk[1] + '_min': 'min', kk[0] + '_' + kk[1] + '_sum': 'sum', kk[0] + '_' + kk[1] + '_std': 'std',
     kk[0] + '_' + kk[1] + '_mean': 'mean'})
df = pd.merge(df, t1, on=kk[0], how='left')

In [ ]:
# 为部分属性列的数据生成新的特征，主要是通过对 V0、V3、V8 和 V12 四个特征进行组合生成新的二元和三元特征。
num_cols = [0, 3, 8, 12]
for i in num_cols:
    for j in num_cols:
        df['new' + str(i) + '*' + str(j)] = df['v_' + str(i)] * df['v_' + str(j)]

for i in num_cols:
    for j in num_cols:
        df['new' + str(i) + '+' + str(j)] = df['v_' + str(i)] + df['v_' + str(j)]

for i in num_cols:
    for j in num_cols:
        df['new' + str(i) + '-' + str(j)] = df['v_' + str(i)] - df['v_' + str(j)]

for i in range(15):
    df['new' + str(i) + '*year'] = df['v_' + str(i)] * df['car_age_year']

在训练模型的过程中，使用了 LightGBMRegressor 作为模型，采用四个参数（'n_estimators'、'learning_rate'、'num_leaves' 和 'lambda_l2'）进行调参。
其中：
'n_estimators'：表示树的数量，设置得越多，模型越复杂，训练得越慢。根据经验，一般取 100 至 10000 之间的数。

'learning_rate'：表示学习率，是一个重要的参数，取值越小，需要的树的数量越多，训练得越慢，但一般能得到更好的性能。

'num_leaves'：表示基分类器的数量，取值越大，模型的复杂度越高，但可能会导致过拟合。

'lambda_l2'：表示 L2 正则化系数，取值越大，正则化效果越强，可以防止过拟合。

在特征工程中，对数据进行了缺失值填充、数据统计和特征组合等操作。这些操作都是为了更好地利用数据中包含的信息，提高模型的性能。

在使用 LightGBMRegressor 对数据进行训练时，还使用了 K-Fold 交叉验证（K=4），通过这种方式可以更准确地评估模型的性能，并防止模型过拟合。

最终得到的结果通过将五次模型训练得到的结果平均作为最终预测结果，并将结果保存到文件中供提交。

In [ ]:
# 使用 LightGBMRegressor 作为模型，对数据进行训练和预测。
# 对数据进行五折交叉检验，最后通过将五次模型训练得到的结果平均作为最终预测结果，并将结果保存到文件中供提交。
df1 = df.copy()
test = df1[df1['price'].isnull()]
X_train = df1[df1['price'].notnull()].drop(['price', 'regDate', 'creatDate', 'SaleID', 'regionCode'], axis=1)
Y_train = df1[df1['price'].notnull()]['price']
X_test = df1[df1['price'].isnull()].drop(['price', 'regDate', 'creatDate', 'SaleID', 'regionCode'], axis=1)
# 五折交叉检验
cols = list(X_train)
oof = np.zeros(X_train.shape[0])
sub = test[['SaleID']].copy()
sub['price'] = 0
feat_df = pd.DataFrame({'feat': cols, 'imp': 0})
skf = KFold(n_splits=4, shuffle=True, random_state=2020)

clf = LGBMRegressor(
    n_estimators=10000,
    learning_rate=0.07,  # 0.02,
    boosting_type='gbdt',
    objective='regression_l1',
    max_depth=-1,
    num_leaves=31,
    min_child_samples=20,
    feature_fraction=0.8,
    bagging_freq=1,
    bagging_fraction=0.8,
    lambda_l2=2,
    random_state=2020,
    metric='mae'
)

mae = 0
for i, (trn_idx, val_idx) in enumerate(skf.split(X_train, Y_train)):
    print('--------------------- 第 {} 折 ---------------------'.format(i + 1))
    trn_x, trn_y = X_train.iloc[trn_idx].reset_index(drop=True), Y_train[trn_idx]
    val_x, val_y = X_train.iloc[val_idx].reset_index(drop=True), Y_train[val_idx]
    clf.fit(
        trn_x, trn_y,
        eval_set=[(val_x, val_y)],
        eval_metric='mae',
        early_stopping_rounds=300,
        verbose=300
    )

    sub['price'] += np.expm1(clf.predict(X_test)) / skf.n_splits
    oof[val_idx] = clf.predict(val_x)
    mae += mean_absolute_error(np.expm1(val_y), np.expm1(oof[val_idx])) / skf.n_splits

print('交叉验证 MAE:', mae)

In [ ]:
# 生成提交文件
sub.to_csv('submit.csv', index=False)